# Web scraper for Spotrac.com 2021 NFL Salary Data

In [1]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [7]:
# Load webpage
r = requests.get("https://www.spotrac.com/nfl/positional/breakdown/")

# Convert to BeautifulSoup object
webpage = bs(r.content)

In [79]:
# Return descendents of table tag. Only one table on page
data_table = webpage.find("table")

# Return table rows
data_rows = data_table.find_all("tr")

34

In [92]:
def get_total(url):
    # Send GET request to defined URL
    r_pos = requests.get(url)
    
    # Convert into beautiful soup object
    pos_webpage = bs(r_pos.content)
    
    table = pos_webpage.find("table")
    footer = table.select("tfoot > tr > td")
    total = float(footer[4].get_text().replace("$", "").replace(",",""))
    cap_share = float(footer[5].get_text().replace("%",""))/100
    return total, cap_share

In [95]:
# Initialize lists
salary_dicts = []
links = []
headers = []

for index,row in enumerate(data_rows[:33]):
    # Create empty dict for team
    salary_info = {}
    
    # Grab table headers from first row of table (positions)
    if index == 0:
        header_list = row.find_all('th')
        for header in header_list:
            headers.append(header.get_text())
    else:
        # Find links to team pages
        table = row.select("tr > td > a")
        links = [link['href'] for link in table[1:]]
        
        # Pull team name from first column
        salary_info["Team"] = table[0].get_text()
        print("Getting data for ", table[0].get_text())
        
        # Pull total and cap_share for each position from team page
        for index, header in enumerate(headers[2:11]):
            print("    Position - ", header)
            header_share = str(header) + "_share"
            salary_info[header], salary_info[header_share] = get_total(links[index])
        
        # Append team dict to full team list
        salary_dicts.append(salary_info)

Getting data for  Arizona Cardinals
    Position -  QB
    Position -  RB/FB
    Position -  WR
    Position -  TE
    Position -  OL
    Position -  DL
    Position -  LB
    Position -  DB
    Position -  K/P/LS
Getting data for  Atlanta Falcons
    Position -  QB
    Position -  RB/FB
    Position -  WR
    Position -  TE
    Position -  OL
    Position -  DL
    Position -  LB
    Position -  DB
    Position -  K/P/LS
Getting data for  Baltimore Ravens
    Position -  QB
    Position -  RB/FB
    Position -  WR
    Position -  TE
    Position -  OL
    Position -  DL
    Position -  LB
    Position -  DB
    Position -  K/P/LS
Getting data for  Buffalo Bills
    Position -  QB
    Position -  RB/FB
    Position -  WR
    Position -  TE
    Position -  OL
    Position -  DL
    Position -  LB
    Position -  DB
    Position -  K/P/LS
Getting data for  Carolina Panthers
    Position -  QB
    Position -  RB/FB
    Position -  WR
    Position -  TE
    Position -  OL
    Position -  D

IndexError: list index out of range

In [96]:
salary_df = pd.DataFrame(salary_dicts)
salary_df

,Team,QB,QB_share,RB/FB,RB/FB_share,WR,WR_share,TE,TE_share,OL,OL_share,DL,DL_share,LB,LB_share,DB,DB_share,K/P/LS,K/P/LS_share
0,Arizona Cardinals,11203721.0,0.0583,3278770.0,0.0171,24919816.0,0.1298,3878750.0,0.0202,52890737.0,0.2754,22615415.0,0.1178,53512960.0,0.2787,16286892.0,0.0848,3243750.0,0.0169
1,Atlanta Falcons,26912500.0,0.1455,5033112.0,0.0272,31032128.0,0.1677,5084913.0,0.0275,25097737.0,0.1357,39615581.0,0.2141,17741178.0,0.0959,11423258.0,0.0617,4359435.0,0.0236
2,Baltimore Ravens,4683880.0,0.0256,6381558.0,0.0349,8240942.0,0.0450,12627939.0,0.0690,30263756.0,0.1653,37247937.0,0.2035,12826269.0,0.0701,39213118.0,0.2142,9320000.0,0.0509
3,Buffalo Bills,11115797.0,0.0590,5161586.0,0.0274,30695400.0,0.1629,3607091.0,0.0191,34846742.0,0.1850,43131714.0,0.2290,24040037.0,0.1276,29412856.0,0.1561,3541054.0,0.0188
4,Carolina Panthers,24888670.0,0.1334,10039500.0,0.0538,21869510.0,0.1173,5222200.0,0.0280,28913847.0,0.1550,13028912.0,0.0699,20121714.0,0.1079,18579752.0,0.0996,2427500.0,0.0130
5,Chicago Bears,11999999.0,0.0631,5991577.0,0.0315,26058590.0,0.1371,13232222.0,0.0696,28237444.0,0.1485,29053309.0,0.1528,43232944.0,0.2274,14210609.0,0.0747,1575000.0,0.0083
6,Cincinnati Bengals,10385031.0,0.0532,15909664.0,0.0815,15773645.0,0.0808,8742551.0,0.0448,27417083.0,0.1405,39455068.0,0.2022,4703021.0,0.0241,39684493.0,0.2034,2644982.0,0.0136
7,Cleveland Browns,18682463.0,0.0882,9246381.0,0.0436,37850277.0,0.1786,17525257.0,0.0827,50288806.0,0.2373,33608993.0,0.1586,5657512.0,0.0267,28339202.0,0.1337,3765834.0,0.0178
8,Dallas Cowboys,24843787.0,0.1183,16280278.0,0.0775,33126755.0,0.1578,7626967.0,0.0363,35508126.0,0.1691,38021856.0,0.1811,16983082.0,0.0809,22025518.0,0.1049,3613333.0,0.0172
9,Denver Broncos,6012041.0,0.0299,14208005.0,0.0708,15878545.0,0.0791,6032262.0,0.0300,34871274.0,0.1737,11478213.0,0.0572,39362155.0,0.1960,24601743.0,0.1225,5869583.0,0.0292
